# Face Alignment

## Выбор датасета

В этом проекте мы будем работать с датасетом [CelebA - Original Wild Images](https://www.kaggle.com/datasets/kevinpatel04/celeba-original-wild-images). Обычно, когда говорят про CelebA имеют ввиду его кропнутую и выровненную версию, но мы будем работать с сырой.

![CelebA](https://figures.semanticscholar.org/7df4f96138a4e23492ea96cf921794fc5287ba72/6-Figure4-1.png)

Оригинальная версия датасета весит очень много: ~20 гб. Все картинки нам не будут нужны, достаточно будет ограничиться 10.000+.

Вашим первым заданием будет подготовить себе рабочий датасет. Он будет использован и в следующих заданиях, поэтому подойдите к этому очень ответственно:

- Скачайте себе датасет CelebA In a Wild любым удобным способом;
- Подумайте над тем, по каким критериям лучше всего выбирать картинки. Используйте файл с атрибутами. Обоснуйте свой выбор текстом. В случае, если обоснования выбора датасета не будет, то баллы могут быть снижены. Отнеситесь к этому серьезно: хорошая подготовка данных очень важна;
- Отберите 10.000+ изображений. Можно и больше при желании;
- При помощи атрибутов bbox'а, обрежьте картинки, чтобы на них остались только лица. При желании можно делать дополнительный кроп, так как не везде разметка идеальна, но не нужно сидеть над каждой картинкой отдельно - потратите слишком много времени;
- Сохраните отдельный csv-файл с оригинальными названиями изображений, которые были отобраны в ваш датасет. В дальнейшем он в том числе понадобится для сдачи проекта.

Несколько практических советов:
- Если решили делать дополнительный кроп изображений самостоятельно, то имейте ввиду, что ключевые точки лица в атрибутах указаны в начальной системе координат.
- Не работайте с датасетом в колабе. Простейшую работу с файлами удобнее всего делать локально на своем компьютере. Это не требует мощного железа и скачивания библиотек. Если все же пользуетесь колабом, то работайте на CPU, чтобы не лишний раз не тратить токены.
- Если есть возможность, то не удаляйте оригинальный датасет с вашего компьютера на период работы над проектом. Да, это лишняя занятая память, но зато при надобности можно будет быстро что-то изменить.
- Загрузите финальный датасет (10.000+ картинок) себе на Google Disk. Это удобнее, чем каждый раз отдельно загружать его себе в колаб сессию.

## Архитектура Stacked Hourglass Network

В разных вариантах пайплайна для распознавания лиц ключевые точки лица могут предсказываться сразу детектором (MTCNN, RetinaFace и прочие), а могут и отдельной моделью. В этом проекте рассматривается второй вариант. То есть, за детекцию ключевых точек будет отвечать отдельная модель.

**Hourglass** — это U-Net-подобная структура, которая сначала уменьшает разрешение изображения, затем восстанавливает его обратно. Такая структура напоминает по форме песочные часы (hourglass).

**Stacked Hourglass Network** состоит Hourglass-блоков, каждый из которых старается уточнять результат предыдущего. Несмотря на то, что она придумана в 2016 году, до сих пор используется во многих исследовательских проектах для задачи обнаружения ключевых точек.

![image](https://img2018.cnblogs.com/blog/900393/201907/900393-20190722093153502-1808128161.png)

### Hourglass module

Посмотрим подробнее на структуру **отдельного Hourglass-блока**

![retrt](https://curt-park.github.io/images/stacked_hourglass_networks/fig3.png)

Каждый бокс в этой схеме - это Residual block, который отвечает за извлечение признаков на разных уровнях детализации (вспоминаем про ResNet). Причем, каждый такой блок имеет одинаковую размерность на входе и на выходе.

Downsampling и upsampling можно делать разными способами.

*   Для Downsampling: nn.MaxPool2d или nn.Conv2d
*   Для Upsampling: nn.Upsample или nn.ConvTranspose2d

Основная разница: maxpool и upsample - необучаемые слои в отличие от сверток. Это может как быть как минусом, так и плюсом: чем больше параметров - тем медленее идет процесс обучения (при этом не факт, что результаты будут лучше).

То есть, идейно все практически также как было в U-net: полностью симметричная архитектура, сначала идет преобразование в более низкоразмерное пространство, а потом декодирование обратно с пробросами результатов из соотвествующих слоев энкодера. Разница лишь в том, что теперь каждый кирпичик - это Residual block.

А вот реализация ResidualBlock вам в помощь!

Но можете ее править под себя, если очень хочется.

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.skip = nn.Identity() if in_channels == out_channels else nn.Conv2d(in_channels, out_channels, 1)

        self.conv1 = nn.Conv2d(in_channels, out_channels // 2, 1)
        self.bn1 = nn.BatchNorm2d(out_channels // 2)
        self.conv2 = nn.Conv2d(out_channels // 2, out_channels // 2, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels // 2)
        self.conv3 = nn.Conv2d(out_channels // 2, out_channels, 1)
        self.bn3 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = self.skip(x)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        return self.relu(x + residual)

При построении архитектуры Hourglass-блоков **не обязательно полностью повторять архитектуру**, которая представлена на картинке из статьи. Вы можете добавлять больше или меньше модулей, некоторые блоки вообще можно не использовать. В целом, это творческая задача и вы вольны делать так, как вам самим хочется. **Главное - чтобы ваша реализация соотвествовала изначальной идее Hourglass, и итоговые результаты были достаточно хорошими.**

### Stacked Hourglass Network

Как и было сказано ранее, Stacked Hourglass - это набор одинаковых Hourglass блоков (см. схематический рисунок в начале ноутбука). Но что это за блок между каждыми двумя Hourglass? Чтобы ответить на этот вопрос, нужно сначала разобраться с тем, что мы будем получать на выходе такой нейронной сети.

Предсказывать ключевые точки лица можно поразному. Есть два основных подхода:

1.   Регрессия - предсказывает координаты точек лица напрямую -> $(N, x, y)$.
2.   Heatmap - предсказывает карту вероятностей на выходе, а наиболее подходящие точки находятся через argmax

Не вдваясь в подробности, можно просто сказать, что Heatmap-подход показал себя лучше из-за своей устойчивости к шумам и начальным условиям. В качестве функции потерь в таком случае используют обычный **MSE loss**.

В Stacked Hourglass **используется именно heatmap-подход**. И на выходе каждого Hourglass-блока находится слой (голова), который создает heatmap нужного размера. Обычно это какие-то стандартные варианты по типу *Conv -> BatchNorm -> Relu -> Conv* или просто *Conv*. Каждая heatmap'a прокидывается на следующую голову, и они суммируются, и так, пока слои не закончатся.

Такой подход нужен для реализации **Intermediate Supervision**. Если говорить простыми словами, то это такой вариант обучения нейронной сети, когда мы подсчитываем лосс не только по финальному выходу сети, а также на некоторых промежуточных слоях (головах). Градиенты в этом случае тоже распространяются не только через последний выход, но и через промежуточные уровни. Эти головы не влияют на финальное предсказание напрямую, но помогают модели быстрее и лучше учиться. На практике это означет следующее:

Нужно посчитать лосс (таргет для всех одинаковый) для каждой головы отдельно, а потом просуммировать. Далее Pytorch сам построит за вас весь граф вычислений и правильно запустит везде градиенты. В коде это выглядит так:

```
outputs = model(image)  # outputs — список из N heatmaps от разных голов
losses = [loss_function(output, target) for output in outputs]
total_loss = sum(losses)
total_loss.backward()
optimizer.step()
```

Подведем **итоги по архитектуре**.

Stacked Hourglass состоит из Hourglass-блоков, после каждого такого блока идет голова, которая предсказывает heatmap'у. Каждая heatmap'а суммируется с предыдущей. Градиенты при обучении текут с каждой головы, а не только через последний выход сети.

Подробно про Stacked Hourglass Network можно прочитать в [оригинальной статье](https://arxiv.org/pdf/1603.06937).

## Подготовка датасета для обучения

На этом этапе у вас уже должен быть готовый датасет на основе CelebA In A Wild.

В разметке CelebA всего 5 точек:

1.   Левый глаз
2.   Правый глаз
3.   Нос
4.   Левый уголок рта
5.   Правый уголок рта


Единственная проблема заключается в том, что разметка - это именно точки, а не heatmap'ы. Но можно их сгенерировать самостоятельно при помощи гауссовского распределения вокруг размеченных точек. Вот вам функции в помощь. Можете их тоже редактивовать под себя, если нужно.

In [ ]:
import numpy as np

def create_heatmap(size, landmark, sigma=2):
    """
    Создаёт один heatmap с гауссовым ядром вокруг точки.

    :param size: (height, width) — размер heatmap'а
    :param landmark:(x, y) — координаты точки
    :param sigma
    :return: heatmap массив
    """
    x, y = landmark
    h, w = size

    # Обрезаем координаты, чтобы не выйти за пределы изображения
    x = min(max(0, int(x)), w - 1)
    y = min(max(0, int(y)), h - 1)

    xx, yy = np.meshgrid(np.arange(w), np.arange(h))
    heatmap = np.exp(-((yy - y)**2 + (xx - x)**2) / (2 * sigma**2))
    return heatmap


def landmarks_to_heatmaps(image_shape, landmarks, sigma=2):
    """
    Преобразует список из N точек в набор из N heatmap'ов.

    :param image_shape: исходный размер изображения (H, W)
    :param landmarks: список из N пар координат [(x1, y1), (x2, y2), ..., (xN, yN),]
    :param sigma:
    :return: массив heatmap'ов вида [N, H, W]
    """
    heatmaps = []

    for (x, y) in landmarks:
        hm = create_heatmap(image_shape, sigma=sigma)
        heatmaps.append(hm)

    return np.array(heatmaps)

## Выравнивание по предсказанным точкам

Существует множество вариантов, как по полученным точкам правильно преобразовать картинку. Главное, что вам нужно понимать - **это задача классического компьютерного зрения** и решается при помощи математики, без нейронок. Вдаваться в подробности конкретных алгоритмов мы не будем.

Можно использовать аффинное преобразование, тогда потребуется только 3 точки, можно, например, искать матрицу гомографии, где может быть использовано больше точек, а может быть, есть еще что-то. Реализовывать эти алгоритмы самим не нужно. Достаточно провести небольшой ресерч и найти готовое решение (но **не готовую нейронку для выравнивания**), лишь бы оно работало. Количество используемых точек тоже выбирайте сами, подойдет любой вариант. Условный ориентир для поиска - библиотека **opencv**. Обязательно приведите примеры того, как работает ваш алгоритм.

# План заданий

По итогу, в этом блоке у вас следующие задачи:

*   Подготовить датасет, сохранить файл с оригинальными названиями изображений
*   Реализовать Hourglass блок
*   Реализовать Stacked Hourglass
*   Преобразовать точки лица в Heatmap'ы
*   Обучить Stacked Hourglass
*   Найти функцию, которая по предсказанным ключевым точкам выравнивает лица на картинке (face alignment)
*   Подготовить датасет с кропнутыми и выровненными лицами для следующего этапа

**P.S. Не забывайте сохранять модели после обучения и выводите промежуточные результаты на экран**



**Удачи! У вас всё получится 💗!**

## Подготовка датасета (Preprocessing)

**Обоснование выбора данных (для отчета)** Критерии отбора 10,000+ изображений:
- *Отсутствие сильных окклюзий (перекрытий)*: фильтровать изображения по атрибутам Wearing_Hat (Шляпа) и Eyeglasses (Очки). Наличие этих предметов может мешать точному определению ключевых точек (глаз).
- *Качество освещения*: Исключим фотографии с атрибутом Blurry (Размытость), чтобы модель обучалась на четких границах органов лица.
- *Размер лица*: будем использовать данные из bbox, выбирая лица, которые занимают значительную часть кадра (площадью не менее $120 \times 120$ пикселей). Это гарантирует, что после кропа картинка не превратится в "набор пикселей".
- *Позиция*: Приоритет отдается фронтальным лицам без экстремальных поворотов (отсутствие атрибута No_Beard часто коррелирует с мужскими лицами в анфас, но просто возьмем первые 10к, прошедшие фильтры по качеству).

In [1]:
import os
import pandas as pd
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

IMG_DIR = 'img_celeba'  # Папка с оригинальными фото (Wild)
SAVE_DIR = 'celeba_cropped_10k'

In [4]:
def read_celeba_txt(path):
    return pd.read_csv(path, sep='\s+', skiprows=1)

attr_df = read_celeba_txt('list_attr_celeba.txt')
bbox_df = read_celeba_txt('list_bbox_celeba.txt').set_index('image_id')
land_df = read_celeba_txt('list_landmarks_celeba.txt').set_index('image_id')
selected_list = pd.read_csv('selected_names.csv', header=None)

<>:2: SyntaxWarning: "\s" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\s"? A raw string is also an option.
<>:2: SyntaxWarning: "\s" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\s"? A raw string is also an option.
C:\Users\mvkiselev\AppData\Local\Temp\ipykernel_67492\692596869.py:2: SyntaxWarning: "\s" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\s"? A raw string is also an option.
  return pd.read_csv(path, sep='\s+', skiprows=1)


In [5]:
selected_list

,0
0,image_id
1,000001.jpg
2,000002.jpg
3,000004.jpg
4,000005.jpg
...,...
10496,011849.jpg
10497,011850.jpg
10498,011851.jpg
10499,011852.jpg


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## Архитектура Stacked Hourglass

Использую предложенный ResidualBlock и строю иерархию.

In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.skip = nn.Identity() if in_channels == out_channels else nn.Conv2d(in_channels, out_channels, 1)
        self.conv1 = nn.Conv2d(in_channels, out_channels // 2, 1)
        self.bn1 = nn.BatchNorm2d(out_channels // 2)
        self.conv2 = nn.Conv2d(out_channels // 2, out_channels // 2, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels // 2)
        self.conv3 = nn.Conv2d(out_channels // 2, out_channels, 1)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bn3(self.conv3(self.relu(self.bn2(self.conv2(self.relu(self.bn1(self.conv1(x)))))))) + self.skip(x))

class Hourglass(nn.Module):
    def __init__(self, depth, channels):
        super().__init__()
        self.depth = depth
        self.up = ResidualBlock(channels, channels)
        self.pool = nn.MaxPool2d(2, 2)
        self.low = Hourglass(depth - 1, channels) if depth > 1 else ResidualBlock(channels, channels)
        self.low_post = ResidualBlock(channels, channels)
        self.up_sample = nn.Upsample(scale_factor=2, mode='nearest')

    def forward(self, x):
        return self.up(x) + self.up_sample(self.low_post(self.low(self.pool(x))))

class StackedHourglass(nn.Module):
    def __init__(self, num_stacks=2, num_pts=5, channels=128):
        super().__init__()
        self.num_stacks = num_stacks
        self.pre = nn.Sequential(nn.Conv2d(3, 64, 7, stride=2, padding=3), nn.BatchNorm2d(64), nn.ReLU(True),
                                 ResidualBlock(64, 128), nn.MaxPool2d(2, 2), ResidualBlock(128, channels))
        self.hgs = nn.ModuleList([Hourglass(4, channels) for _ in range(num_stacks)])
        self.features = nn.ModuleList([nn.Sequential(ResidualBlock(channels, channels), nn.Conv2d(channels, channels, 1),
                                                    nn.BatchNorm2d(channels), nn.ReLU(True)) for _ in range(num_stacks)])
        self.outs = nn.ModuleList([nn.Conv2d(channels, num_pts, 1) for _ in range(num_stacks)])
        self.merge_preds = nn.ModuleList([nn.Conv2d(num_pts, channels, 1) for _ in range(num_stacks-1)])
        self.merge_feats = nn.ModuleList([nn.Conv2d(channels, channels, 1) for _ in range(num_stacks-1)])

    def forward(self, x):
        x = self.pre(x)
        outputs = []
        for i in range(self.num_stacks):
            hg = self.hgs[i](x)
            feature = self.features[i](hg)
            preds = self.outs[i](feature)
            outputs.append(preds)
            if i < self.num_stacks - 1:
                x = x + self.merge_feats[i](feature) + self.merge_preds[i](preds)
        return outputs

## Генерация Heatmaps и Dataset

In [7]:
class CelebADataset(Dataset):
    def __init__(self, csv_file, land_df, bbox_df, img_dir):
        self.names = pd.read_csv(csv_file)['image_id'].values
        self.land_df, self.bbox_df, self.img_dir = land_df, bbox_df, img_dir

    def __len__(self): return len(self.names)

    def __getitem__(self, idx):
        name = self.names[idx]
        img = cv2.cvtColor(cv2.imread(os.path.join(self.img_dir, name)), cv2.COLOR_BGR2RGB)

        # Пересчет координат (ресайз 128x128)
        b = self.bbox_df.loc[name]
        m_w, m_h = int(b['width']*0.1), int(b['height']*0.1)
        orig_pts = self.land_df.loc[name].values.reshape(5, 2)

        # Сдвиг и масштаб
        new_pts = (orig_pts - [b['x_1'] - m_w, b['y_1'] - m_h]) * [128/(b['width']+2*m_w), 128/(b['height']+2*m_h)]

        # Heatmaps (32x32)
        hms = []
        for pt in new_pts:
            x, y = pt / 4.0 # так как выход сети 32x32
            xx, yy = np.meshgrid(np.arange(32), np.arange(32))
            hms.append(np.exp(-((yy - y)**2 + (xx - x)**2) / (2 * 1.5**2)))

        return torch.tensor(img).permute(2,0,1).float()/255., torch.tensor(np.array(hms)).float()

## Обучить Stacked Hourglass и Alignment

In [11]:
def calculate_pck(pred_hms, target_hms, threshold=0.1):
    """
    Расчет PCK (Percentage of Correct Keypoints).
    pred_hms, target_hms: тензоры [Batch, 5, 32, 32]
    threshold: порог в долях от размера heatmap (0.1 от 32 пикселей = 3.2 пикселя)
    """
    batch_size, num_pts, h, w = pred_hms.shape
    
    # 1. Находим координаты (x, y) через argmax
    def get_coords(hms):
        hms_flat = hms.view(batch_size, num_pts, -1)
        max_idx = torch.argmax(hms_flat, dim=2)
        preds = torch.zeros((batch_size, num_pts, 2))
        preds[:, :, 0] = max_idx % w  # x
        preds[:, :, 1] = max_idx // w # y
        return preds

    preds = get_coords(pred_hms)
    targets = get_coords(target_hms)

    # 2. Считаем Евклидово расстояние между точками
    # dist: [Batch, 5]
    dist = torch.sqrt(torch.sum((preds - targets)**2, dim=2))
    
    # Порог (например, 10% от ширины heatmap)
    limit = threshold * max(h, w)
    
    # 3. Считаем, сколько точек попало в порог
    correct = (dist <= limit).float()
    
    # Возвращаем среднюю точность по всему батчу и всем точкам
    return correct.mean().item()

In [20]:
selected_df = pd.read_csv('selected_names.csv')
selected_list = selected_df['image_id'].tolist()

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = StackedHourglass().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
loader = DataLoader(CelebADataset('selected_names.csv', land_df, bbox_df, SAVE_DIR), batch_size=32, shuffle=True)

# Обучение
for epoch in range(10):
    model.train()
    total_loss = 0.0
    running_pck = 0.0
    for imgs, targets in tqdm(loader, desc=f"Epoch {epoch}"):
        imgs, targets = imgs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = sum(criterion(o, targets) for o in outputs)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Считаем точность по последнему (самому точному) стеку
        current_pck = calculate_pck(outputs[-1].detach(), targets)
        running_pck += current_pck
        
    print(f"Loss: {total_loss/len(loader)} PCK@0.1: {running_pck / len(loader)}")
torch.save(model.state_dict(), 'hourglass.pth')

# Face Alignment Функция
def align_face(img, pts):
    # Эталонные точки (глаза и нос)
    dst = np.array([[35, 45], [93, 45], [64, 75]], dtype=np.float32)
    src = pts[:3].astype(np.float32)
    M = cv2.getAffineTransform(src, dst)
    return cv2.warpAffine(img, M, (128, 128))



Epoch 0: 100%|██████████| 329/329 [02:45<00:00,  1.99it/s]


Loss: 0.02643428921507032 PCK@0.1: 0.5630699101054197


Epoch 1: 100%|██████████| 329/329 [02:48<00:00,  1.95it/s]


Loss: 0.007090132900069278 PCK@0.1: 0.9464665638639572


Epoch 2: 100%|██████████| 329/329 [02:50<00:00,  1.93it/s]


...

Loss: 0.0012412520191323658 PCK@0.1: 0.9937499982607764


Epoch 8: 100%|██████████| 329/329 [02:47<00:00,  1.97it/s]


Loss: 0.0011082699598099096 PCK@0.1: 0.9943768978118896


Epoch 9: 100%|██████████| 329/329 [02:49<00:00,  1.94it/s]

Loss: 0.0010071369224174181 PCK@0.1: 0.9948328258781085


In [23]:
# Финальный шаг: подготовка выровненного датасета
ALIGNED_DIR = 'celeba_aligned'
os.makedirs(ALIGNED_DIR, exist_ok=True)

model.eval()
selected_df = pd.read_csv('selected_names.csv')
selected_list = selected_df['image_id'].tolist()

print(f"Начинаю выравнивание {len(selected_list)} изображений...")

with torch.no_grad():
    for img_id in tqdm(selected_list):
        img_path = os.path.join(SAVE_DIR, str(img_id))
        img_raw = cv2.imread(img_path)
        
        if img_raw is None: continue
            
        # Подготовка тензора для модели
        img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
        input_t = torch.tensor(img_rgb).permute(2, 0, 1).float().unsqueeze(0).to(device) / 255.0
        
        # Предсказание (берем последний стек)
        output_hms = model(input_t)[-1].cpu().numpy()[0]
        
        # Декодируем точки из Heatmap
        pts = []
        for i in range(5):
            hm = output_hms[i]
            y, x = np.unravel_index(np.argmax(hm), hm.shape)
            # Умножаем на 4, так как heatmap 32x32, а картинка 128x128
            pts.append([x * 4, y * 4])
        
        pts = np.array(pts)
        
        # Применяем выравнивание
        aligned_face_img = align_face(img_raw, pts)
        
        # Сохраняем результат
        cv2.imwrite(os.path.join(ALIGNED_DIR, img_id), aligned_face_img)

print("Выравнивание завершено!")

Начинаю выравнивание 10500 изображений...


100%|██████████| 10500/10500 [01:25<00:00, 122.61it/s]

Выравнивание завершено!
